In [36]:
import numpy as np

s = 'hi how are you'
t = np.array(list(s.encode('utf8')))

r = t*sum(t)
r = r / sum(r)
r = r * 16384
r 

array([1310.72      , 1323.32307692,  403.29846154, 1310.72      ,
       1398.94153846, 1499.76615385,  403.29846154, 1222.49846154,
       1436.75076923, 1272.91076923,  403.29846154, 1524.97230769,
       1398.94153846, 1474.56      ])